In [1]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

In [3]:
!nvidia-smi

Tue Oct 31 21:46:17 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.04   Driver Version: 525.116.04   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100 80G...  Off  | 00000000:00:05.0 Off |                    0 |
| N/A   44C    P0    72W / 300W |  11707MiB / 81920MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
all_df = pd.read_parquet('all_train_df.parquet')
all_df.head(5)

,Unnamed: 0,community_id,description,customers_count,messages_count,type,region_id,themeid,business_category,business_parent,customer_id,status,join_request_date,count_db,pair
15051500,15051500,d501adec049a83d7e968b9836d55d212e952e157fe4750...,False,6429,1,7,1.040205e+10,234.0,CARS_MACHINERY,AUTO,1a4bd984b1a1c6ef217dafdbd4c69734c2ecc785ae87a2...,I,NaN,1,1a4bd984b1a1c6ef217dafdbd4c69734c2ecc785ae87a2...
8983316,8983316,2e1b38e5b2798ad629fbcdc65e753ac381a50e7ab0b059...,False,50674,1768,7,NaN,NaN,COOKING_AND_RECIPES,FOOD,fe11157c925c62cd0538220d613a4bc450a064cb83d31a...,A,NaN,2,fe11157c925c62cd0538220d613a4bc450a064cb83d31a...
7000208,7000208,992a757cb164d3f767295af497039ad6cc3e3fa091b42a...,False,6030,1,7,NaN,NaN,COOKING_AND_RECIPES,FOOD,1bd41531bace0d2d228797f2e0bfde305ba34f7f5920a7...,A,NaN,2,1bd41531bace0d2d228797f2e0bfde305ba34f7f5920a7...
10065277,10065277,5189397d7058464aff8179b792dfda693c8cd9d09bd8b5...,True,322968,91,7,NaN,392.0,COOKING_AND_RECIPES,FOOD,944f31e05522ec9b6ae05f6acac919009af9f2e4266018...,P,NaN,1,944f31e05522ec9b6ae05f6acac919009af9f2e4266018...
13824793,13824793,56d54d70c65fae5f81f8db9267a3b58698ad18ab070831...,False,1681,2,7,1.039461e+10,NaN,OTHER_SERVICES,PROFESSIONAL_SERVICES,83d52f013852c463b478507f2397656aba59d666c3884e...,P,NaN,1,83d52f013852c463b478507f2397656aba59d666c3884e...


In [5]:
als_recs = pd.read_parquet('train_gbm.parquet')
als_recs.head(5)

,query,user_id,scor,rank,comunity,label
0,0,00005f5e3d17a1458728b9d17dfbc5167ecb33696b1253...,0.656917,0,f3f86ee64f63f549e242ac524a19a349a3c953907064d2...,0
1,0,00005f5e3d17a1458728b9d17dfbc5167ecb33696b1253...,0.584977,1,0e8a2d7022005f85b97b19284423e6f19dfac1761bab5a...,0
2,0,00005f5e3d17a1458728b9d17dfbc5167ecb33696b1253...,0.550016,2,eb19a386a1fb1dd679cf7d633d81e347c8f0aa8f04df4f...,0
3,0,00005f5e3d17a1458728b9d17dfbc5167ecb33696b1253...,0.511569,3,93f4db7d1d4bcb0169b46f8b7c823b786e913d2b27f252...,0
4,0,00005f5e3d17a1458728b9d17dfbc5167ecb33696b1253...,0.480877,4,b02102475eb638889c66ac35b21704d6cc1a75db2be8f6...,0


In [6]:
user_tr_group = all_df[['customer_id','community_id']].groupby('customer_id').agg(lambda x:x.tolist())

## User Features

In [7]:
user_feat = all_df[['customer_id','customers_count','messages_count','type',
                    'themeid','business_category','business_parent']].groupby('customer_id').agg(lambda x:x.tolist())

In [8]:
user_feat

,customers_count,messages_count,type,themeid,business_category,business_parent
customer_id,,,,,,
000004017025a08c984916e57658b1ab64deb4172db1634f6acf5c583bd900c15a8f9281a2404cd9a658eda02cdd849964d86892dd22a6435758383520f64d8c,"[198474, 120967, 68248, 56227, 157082, 279299,...","[12, 13, 13, 10, 12, 2, 14, 10, 14, 13, 56]","[7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]","[320.0, 275.0, 135.0, 425.0, 49.0, 248.0, nan,...","[COOKING_AND_RECIPES, COOKING_AND_RECIPES, COO...","[FOOD, FOOD, FOOD, FOOD, FOOD, None, FOOD, FOO..."
0000079a765e2fd7fbb3d8c6968474d7a40cc9ada4c9ec430e3e145cac132150e34b54ed01393d202987a80999901b2a5d2017aae5d087a3241c1e4f3ceed6db,"[696, 26713, 55692, 39508, 30863, 71127, 74161...","[1, 48, 6, 55, 741, 65, 4, 50, 2, 9, 81, 51, 2]","[7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]","[nan, 360.0, 235.0, nan, 248.0, nan, 310.0, na...","[None, ANIMALS, FAMILY_CHILDREN, CREATION, FAM...","[None, PETS, BLOG, BLOG, BLOG, BLOG, BLOG, BLO..."
000015dccbb94f6926bfedc1578d6207ab10c6580fb121c0cd1a70250cd06fc4bae36468143d664865b7a9e63e34b894824a92646e01eb06e028593eeb0ac868,[948],[5],[7],[nan],[RELIGIOUS_INSTITUTION],[SOCIAL_ORGANISATIONS]
00001a206a3508a172a7be2b19c450b72adfc46239e1ed65bf49b8aadb6106b395769d8d4b8b1a1c20eed4cb0de77f6605ca30602223d143082ccca717fc4903,[265],[1],[7],[763.0],[GOS_HIGHER_EDUCATION],[GOS_ORGANIZATIONS]
0000208fc1af439a84a92acd34352268ab169b1a6536e6b51960cda8af910098c4aded0511e7e44ef649aec480eca447af7d86c7f3f0e3ac23c542c64fbc4e47,[133799],[7],[7],[286.0],[CULTURE_AND_ART],[BLOG]
...,...,...,...,...,...,...
ffffe958dc1b390cf49fa987bb1b170f73f0e56a78a936144a19a1ab750d88b154fad72335200731010ba0d1d842fc7105eaaa3033e300cc745522c9dacaa595,[115355],[548],[7],[373.0],[None],[None]
ffffea23f1b7986780b338ea84175e35790ae9b2e3688eed9f4709e41dabbedf67e6f2184f401172c0f43ae2579bf5a4c45fe5a3efe8d1db2c3f7bc6e7b548a6,"[153349, 802708, 4249, 4116, 293090, 5549]","[39, 243, 4, 1, 17, 7]","[7, 7, 7, 7, 7, 7]","[839739007730.0, 44.0, 767.0, nan, 318.0, 360.0]","[RELIGIOUS_INSTITUTION, COOKING_AND_RECIPES, J...","[SOCIAL_ORGANISATIONS, FOOD, PERSON, PERSON, B..."
ffffea67894044381d713871f6cb6d482a45c031b1272f169b3facd9d64c94c4ca8023bff722c34cfd5ac725563d066316029d1d47e83d2de7ed2a20d1c11cfd,"[1282, 9028, 55992, 7924, 252502, 68060, 10521...","[2, 3, 1, 3, 2, 5, 27, 1, 4, 1, 4, 4, 3, 1]","[7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]","[734.0, 205.0, 763.0, 73.0, 51.0, 861186632026...","[GOS_SCHOOL, None, None, None, SPORTS_ORGANIZA...","[GOS_ORGANIZATIONS, None, None, None, SPORT, A..."


In [9]:
user_feat['mean_customers'] = user_feat['customers_count'].apply(np.mean)
user_feat['min_customers'] = user_feat['customers_count'].apply(np.min)
user_feat['max_customers'] = user_feat['customers_count'].apply(np.max)
user_feat['std_customers'] = user_feat['customers_count'].apply(np.std)

user_feat['mean_mess_count'] = user_feat['messages_count'].apply(np.mean)
user_feat['min_mess_count'] = user_feat['messages_count'].apply(np.min)
user_feat['max_mess_count'] = user_feat['messages_count'].apply(np.max)
user_feat['std_mess_count'] = user_feat['messages_count'].apply(np.std)

In [10]:
user_feat['type_ft'] = user_feat['type'].apply(lambda x:x[0])

In [11]:
import collections
def most_count(lst,top=1):
    cmn = collections.Counter(lst).most_common()
    if len(cmn) < top:
        return 'NuN'
    else:
        return cmn[top-1][0]

In [12]:
user_feat['bs_ct_top1'] = user_feat['business_category'].apply(lambda x:most_count(x,top=1))
user_feat['bs_ct_top2'] = user_feat['business_category'].apply(lambda x:most_count(x,top=2))
user_feat['bs_ct_top3'] = user_feat['business_category'].apply(lambda x:most_count(x,top=3))

In [13]:
user_feat['bp_ct_top1'] = user_feat['business_parent'].apply(lambda x:most_count(x,top=1))
user_feat['bp_ct_top2'] = user_feat['business_parent'].apply(lambda x:most_count(x,top=2))
user_feat['bp_ct_top3'] = user_feat['business_parent'].apply(lambda x:most_count(x,top=3))

In [14]:
user_feat['bp_th_top1'] = user_feat['themeid'].apply(lambda x:most_count(x,top=1))
user_feat['bp_th_top2'] = user_feat['themeid'].apply(lambda x:most_count(x,top=2))
user_feat['bp_th_top3'] = user_feat['themeid'].apply(lambda x:most_count(x,top=3))

## Item Features

In [15]:
item_feat = all_df[['community_id','customers_count','messages_count','type',
                    'business_category','business_parent','themeid']].groupby('community_id').agg(lambda x:x.tolist()[0])

In [16]:
item_feat

,customers_count,messages_count,type,business_category,business_parent,themeid
community_id,,,,,,
00000fe7454923588a3e0969bd6fdb2ded31d0ec2e9affa3ff01a26abe8a04dd5aac695ce7c7503f41ce49c1ac88952cbf09f22c6c18b06cca1dae5a9fc24f4e,392,2,7,CREATION,BLOG,309.0
0000656b445bce94e3fc0aa48555767e18bf90dfed755cdafb9fdac64f78c3ee5667f214c6971327c7751a73845cd7bc54824d18e5ef96c35ce73e16e64f4c12,132,1,7,None,None,NaN
000083bb7213fa459fd3b1f7cf17d70d77b4e2af082c82b7e4e2423d69b064a8cadbe7a4a08748fda4b1f5a480861178cb34b553a6a9755c508fdb04eca2c755,168,1,7,None,None,NaN
0000e6f9ebc5b70a58f8a445fe881220e10a0977a7c11b44e4cda378f30d858aadd5cb6b1d4eb40ed4c075578518d1f178aff653b4852257e7d62121e7f9bbb2,32876,4,7,HUMOR,BLOG,NaN
0001895f57e8ef1002c00c2598d4548c8ee911859a6a15a1efd6d59f6649c93be0e1b355cbfb908edd0f78bd827c5968dbdc7b8444abd9f14c926205819c0d23,425,1,7,SCIENCE,SOCIAL_ORGANISATIONS,NaN
...,...,...,...,...,...,...
fffcb9862b47edd9f1ddfa61187f6fc199a6177032fa25808486b6c5d748a16cb93c7742a92eb439f366af0938d0e6ee134854ba286615af90e17312961fb902,115,2,7,GOS_ORGANIZATION,GOS_ORGANIZATIONS,NaN
fffcbfef04d56e028bf7652eae01668467ab6960447fa596e2bf7251578edaa7edda63bed83cc994d52b567ca9ba8f193b79c0b4294dcf1000dbd8285c064741,1877,3,7,None,None,323.0
fffd2b20275f3a3dd5ac58234c30935fd8808217a17a9872d150ac088216b989bcb766de6580f88e5e8ee30ef03f6cac5d6281a4d844753d139d509b3e157c37,122,2,7,BLOGGER,PERSON,NaN


## Merging

In [17]:
item_feat['comunity'] = item_feat.index

In [18]:
user_feat['user_id'] = user_feat.index

In [19]:
als_recs = als_recs.merge(item_feat,how='inner', on='comunity')

In [20]:
als_recs = als_recs.merge(user_feat[['mean_customers','mean_mess_count','type_ft',
                                                       'bs_ct_top1','bs_ct_top2','bs_ct_top3',
                                                       'bp_ct_top1','bp_ct_top2','bp_ct_top3','user_id',
                                                       'bp_th_top1','bp_th_top2','bp_th_top3',
                                                       'std_customers','min_customers','max_customers',
                                                       'std_mess_count','min_mess_count','max_mess_count']],
                                            how='inner', 
                                            on='user_id')

In [21]:
als_recs['bp_th_top1'] =  als_recs['bp_th_top1'].apply(lambda x: -1 if type(x) == str else x)
als_recs['bp_th_top2'] =  als_recs['bp_th_top2'].apply(lambda x: -1 if type(x) == str else x)
als_recs['bp_th_top3'] =  als_recs['bp_th_top3'].apply(lambda x: -1 if type(x) == str else x)

In [22]:
als_recs.to_parquet('all_train_gbm.parquet')

In [23]:
test_recs = pd.read_parquet('test_gbm.parquet')

In [24]:
test_recs = test_recs.merge(item_feat,how='inner', on='comunity')

In [25]:
test_recs = test_recs.merge(user_feat[['mean_customers','mean_mess_count','type_ft',
                                                       'bs_ct_top1','bs_ct_top2','bs_ct_top3',
                                                       'bp_ct_top1','bp_ct_top2','bp_ct_top3','user_id',
                                                       'bp_th_top1','bp_th_top2','bp_th_top3',
                                                       'std_customers','min_customers','max_customers',
                                                       'std_mess_count','min_mess_count','max_mess_count']],
                                            how='inner', 
                                            on='user_id')

In [26]:
test_recs['bp_th_top1'] =  test_recs['bp_th_top1'].apply(lambda x: -1 if type(x) == str else x)
test_recs['bp_th_top2'] =  test_recs['bp_th_top2'].apply(lambda x: -1 if type(x) == str else x)
test_recs['bp_th_top3'] =  test_recs['bp_th_top3'].apply(lambda x: -1 if type(x) == str else x)

In [27]:
test_recs.to_parquet('all_test_gbm.parquet')

In [28]:
pred_recs = pd.read_parquet('pred_gbm.parquet')
pred_recs = pred_recs.merge(item_feat,how='inner', on='comunity')
pred_recs = pred_recs.merge(user_feat[['mean_customers','mean_mess_count','type_ft',
                                                       'bs_ct_top1','bs_ct_top2','bs_ct_top3',
                                                       'bp_ct_top1','bp_ct_top2','bp_ct_top3','user_id',
                                                       'bp_th_top1','bp_th_top2','bp_th_top3',
                                                       'std_customers','min_customers','max_customers',
                                                       'std_mess_count','min_mess_count','max_mess_count']],
                                            how='inner', 
                                            on='user_id')
pred_recs['bp_th_top1'] =  pred_recs['bp_th_top1'].apply(lambda x: -1 if type(x) == str else x)
pred_recs['bp_th_top2'] =  pred_recs['bp_th_top2'].apply(lambda x: -1 if type(x) == str else x)
pred_recs['bp_th_top3'] =  pred_recs['bp_th_top3'].apply(lambda x: -1 if type(x) == str else x)
pred_recs.to_parquet('all_pred_gbm.parquet')

In [30]:
pred_recs

,query,user_id,scor,rank,comunity,customers_count,messages_count,type,business_category,business_parent,...,bp_ct_top3,bp_th_top1,bp_th_top2,bp_th_top3,std_customers,min_customers,max_customers,std_mess_count,min_mess_count,max_mess_count
0,0,d811a09d435ac2d1b1ed46e272405af10933b4711f4564...,0.760456,0,768680a00db384987ff7532abc1ebc8062281b4ac60124...,35518,2,7,FILM,PERSON,...,BLOG,NaN,NaN,62.0,18525.650359,182,73843,7.062444,1,30
1,0,d811a09d435ac2d1b1ed46e272405af10933b4711f4564...,0.534067,1,390a9c1054f80353840567b156cb0b703d3c2a963a5d70...,21676,1,7,INTERNET_MEDIA,MEDIA,...,BLOG,NaN,NaN,62.0,18525.650359,182,73843,7.062444,1,30
2,0,d811a09d435ac2d1b1ed46e272405af10933b4711f4564...,0.481027,2,6b1a156739da983b1d56d32cc8cb128b5b2a1c829de63c...,13858,14,7,TOURIST_AGENCY,TOURISM,...,BLOG,NaN,NaN,62.0,18525.650359,182,73843,7.062444,1,30
3,0,d811a09d435ac2d1b1ed46e272405af10933b4711f4564...,0.440715,3,4ec66521f946fe87d1f3d75e5ef39e01fb6b89996fbe8d...,13206,9,7,LAW_OFFICE,PROFESSIONAL_SERVICES,...,BLOG,NaN,NaN,62.0,18525.650359,182,73843,7.062444,1,30
4,0,d811a09d435ac2d1b1ed46e272405af10933b4711f4564...,0.386041,4,53bbe4405c6abfe03092a641607e70637bfafdf420b088...,11226,7,7,ANIMALS,PETS,...,BLOG,NaN,NaN,62.0,18525.650359,182,73843,7.062444,1,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3452295,5302,28e5354f7ffab3c1238dbf1b17bb2397d511b71ced4ea6...,0.076348,10,c03bf9aa00eb488975523a073ed4104a26f22e4d990886...,23280,2,7,BLOGGER,PERSON,...,SOCIAL_ORGANISATIONS,NaN,NaN,NaN,1083.512547,261,3043,1.200000,1,4
3452296,5302,28e5354f7ffab3c1238dbf1b17bb2397d511b71ced4ea6...,0.073375,23,6651ca2f3d0b27d0fe06bab11caa4fad73ee6a91613c3a...,9699,3,7,PUBLIC_ORGANIZATION,SOCIAL_ORGANISATIONS,...,SOCIAL_ORGANISATIONS,NaN,NaN,NaN,1083.512547,261,3043,1.200000,1,4
3452297,5302,28e5354f7ffab3c1238dbf1b17bb2397d511b71ced4ea6...,0.074058,21,2de900faf69694fdb245d99881502d8539b590f1126958...,10129,0,7,CARS_MACHINERY,AUTO,...,SOCIAL_ORGANISATIONS,NaN,NaN,NaN,1083.512547,261,3043,1.200000,1,4
3452298,5302,28e5354f7ffab3c1238dbf1b17bb2397d511b71ced4ea6...,0.071528,38,a93c51bf7a218a97d04167b943502fdaebcca683c9bb7c...,9401,0,7,CAMPING,LEISURE_AND_ENTERTAINMENT,...,SOCIAL_ORGANISATIONS,NaN,NaN,NaN,1083.512547,261,3043,1.200000,1,4
